In [8]:
import gym
from gym import spaces
import numpy as np
import pygame
import random
import sys

In [9]:
class FlappyBirdEnv(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(self):
        super(FlappyBirdEnv, self).__init__()

        # Display settings
        self.WIDTH = 400
        self.HEIGHT = 600
        self.FPS = 30

        # Bird settings
        self.bird_x = 50
        self.bird_y = self.HEIGHT // 2
        self.bird_radius = 15
        self.gravity = 1.5
        self.flap_power = -10
        self.bird_vel = 0

        # Pipe settings
        self.pipe_width = 60
        self.pipe_gap = 150
        self.pipe_vel = 4
        self.pipes = []

        # Time and reward
        self.score = 0
        self.timestep = 0

        # Gym spaces
        high = np.array([self.HEIGHT, 1000, self.HEIGHT], dtype=np.float32)
        low = np.array([0, -1000, 0], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)
        self.action_space = spaces.Discrete(2)

        # Pygame rendering
        self.screen = None
        self.clock = None
        self.isopen = True

    def reset(self):
        self.bird_y = self.HEIGHT // 2
        self.bird_vel = 0
        self.score = 0
        self.timestep = 0
        self.pipes = []
        self._spawn_pipe()
        return self._get_obs()

    def step(self, action):
        self.timestep += 1

        # Apply action
        if action == 1:
            self.bird_vel = self.flap_power

        # Bird movement
        self.bird_vel += self.gravity
        self.bird_y += self.bird_vel

        # Move pipes
        for pipe in self.pipes:
            pipe['x'] -= self.pipe_vel

        # Remove off-screen pipes and spawn new ones
        if self.pipes[0]['x'] + self.pipe_width < 0:
            self.pipes.pop(0)
            self._spawn_pipe()
            self.score += 1

        # Collision detection
        done = False
        reward = 1.0  # Living reward

        if self.bird_y < 0 or self.bird_y > self.HEIGHT:
            done = True
            reward = -100.0

        pipe = self.pipes[0]
        if self._collides(pipe):
            done = True
            reward = -100.0

        obs = self._get_obs()
        return obs, reward, done, {}

    def render(self, mode="human"):
        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
            pygame.display.set_caption("Flappy Bird Custom Env")
            self.clock = pygame.time.Clock()

        self.screen.fill((135, 206, 250))  # Sky blue

        # Draw pipes
        for pipe in self.pipes:
            pygame.draw.rect(self.screen, (0, 255, 0), (pipe['x'], 0, self.pipe_width, pipe['gap_start']))
            pygame.draw.rect(self.screen, (0, 255, 0), (pipe['x'], pipe['gap_start'] + self.pipe_gap, self.pipe_width, self.HEIGHT))

        # Draw bird
        pygame.draw.circle(self.screen, (255, 255, 0), (self.bird_x, int(self.bird_y)), self.bird_radius)

        pygame.display.update()
        self.clock.tick(self.FPS)

    def close(self):
        if self.screen is not None:
            pygame.quit()
            self.isopen = False

    def _get_obs(self):
        pipe = self.pipes[0]
        return np.array([
            self.bird_y,
            self.bird_vel,
            pipe['gap_start'] + self.pipe_gap / 2 - self.bird_y
        ], dtype=np.float32)

    def _spawn_pipe(self):
        gap_start = random.randint(100, self.HEIGHT - self.pipe_gap - 100)
        pipe = {
            'x': self.WIDTH,
            'gap_start': gap_start
        }
        self.pipes.append(pipe)

    def _collides(self, pipe):
        if pipe['x'] < self.bird_x + self.bird_radius < pipe['x'] + self.pipe_width:
            if self.bird_y < pipe['gap_start'] or self.bird_y > pipe['gap_start'] + self.pipe_gap:
                return True
        return False

In [10]:
env = FlappyBirdEnv()
obs = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    obs, reward, done, _ = env.step(action)
    env.render()

env.close()

In [11]:
!jupyter nbconvert --to script flappy_custom_env.ipynb

[NbConvertApp] Converting notebook flappy_custom_env.ipynb to script
[NbConvertApp] Writing 8886 bytes to flappy_custom_env.py
